In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import ndcg_score

from sentence_transformers import SentenceTransformer
import warnings

warnings.filterwarnings("ignore")  # 경고 메시지 숨김

# =========================
# 여기만 네 환경에 맞게 수정
# =========================
CSV_PATH = r"C:\work\Project\sports-analysis-fighter\csv_파일\찐최종.csv"

TEXT_COL = "query"
GROUP_COL = "query"
RANK_COL = "llm_rank"
LEAGUE_COL = "추천 리그"

# 팀 컬럼도 피처로 쓰고 싶으면 True
USE_TEAM_COLS = True
TEAM_COLS = ["original_support_team", "matching_team"]

# 수치형 점수(스케일링 대상)
SCORE_COLS = ["sbert_score", "n2v_score", "vector_score"]

# SBERT + PCA + KMeans 설정
SBERT_MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
PCA_DIM = 5
N_CLUSTERS = 5

# (옵션) TF-IDF + SVD 추가할지
USE_TFIDF_SVD = True
TFIDF_MAX_FEATURES = 5000
SVD_DIM = 50

# LGBMRanker 파라미터
LGB_PARAMS = dict(
    objective="lambdarank",
    metric="ndcg",
    learning_rate=0.05,
    max_depth=5,
    n_estimators=300,
    random_state=42,
    importance_type="gain",
)


c:\Users\ghks0\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Step 1: 데이터 로드 및 기본 전처리 진행 중...")

df = pd.read_csv(CSV_PATH)

# 1) 핵심 컬럼 결측 제거
df = df.dropna(subset=[TEXT_COL, GROUP_COL, RANK_COL]).copy()

# 2) 텍스트는 문자열 보장
df[TEXT_COL] = df[TEXT_COL].astype(str)

# 3) 수치형 컬럼 숫자 변환 (문자 섞이면 NaN)
for c in SCORE_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=SCORE_COLS).copy()

# 4) 타겟(relevance) 만들기
#    llm_rank(1이 최고)를 "값이 클수록 좋은 점수"로 바꾸기
#    ⚠️ 반드시 질문 그룹 내부에서 변환해야 랭킹 문제에 맞음
df["relevance"] = df.groupby(GROUP_COL)[RANK_COL].transform(lambda s: s.max() - s + 1)

print("데이터 크기:", df.shape)
df.head(3)


Step 1: 데이터 로드 및 기본 전처리 진행 중...
데이터 크기: (8857, 13)


,user_type,original_support_team,query,matching_team,recommend_league,sbert_score,n2v_score,vector_score,match_score,llm_confidence,llm_rank,llm_reason,relevance
0,1,뉴캐슬 유나이티드,나는 뉴캐슬 유나이티드의 '70년 무관 탈출' 같은 스타일이 마음에 들어. 이런 느...,레이싱 불스,F1,0.448592,0.500000,0.924608,0.5644,0.0,0,분석 실패 또는 신뢰도 낮음,3
1,1,뉴캐슬 유나이티드,나는 뉴캐슬 유나이티드의 '70년 무관 탈출' 같은 스타일이 마음에 들어. 이런 느...,페라리,F1,0.354678,0.572448,0.867586,0.5444,0.0,0,분석 실패 또는 신뢰도 낮음,3
2,1,뉴캐슬 유나이티드,나는 뉴캐슬 유나이티드의 '70년 무관 탈출' 같은 스타일이 마음에 들어. 이런 느...,레드불,F1,0.389818,0.500000,0.869819,0.5299,0.0,0,분석 실패 또는 신뢰도 낮음,3


In [3]:
print("Step 2: 질문(Group) 기준 Train/Test 분리 진행 중...")

X_dummy = df[[TEXT_COL]]
y_dummy = df["relevance"]
groups = df[GROUP_COL]

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_dummy, y_dummy, groups))

train_df = df.iloc[train_idx].copy()
test_df = df.iloc[test_idx].copy()

# group 배열이 행 순서에 의존하므로 질문 기준 정렬해두면 안전함
train_df = train_df.sort_values(GROUP_COL).copy()
test_df = test_df.sort_values(GROUP_COL).copy()

print("train:", train_df.shape, " / test:", test_df.shape)


Step 2: 질문(Group) 기준 Train/Test 분리 진행 중...
train: (7130, 13)  / test: (1727, 13)


In [4]:
print("Step 3: SBERT 임베딩 및 PCA 진행 중...")

sbert = SentenceTransformer(SBERT_MODEL_NAME)

# 질문 유니크를 train/test로 따로 뽑음
train_questions = train_df[TEXT_COL].unique()
test_questions = test_df[TEXT_COL].unique()

# 1) 문장 -> 임베딩 벡터
train_emb = sbert.encode(train_questions)
test_emb = sbert.encode(test_questions)

# 2) PCA는 train으로만 fit
pca = PCA(n_components=PCA_DIM, random_state=42)
train_pca = pca.fit_transform(train_emb)
test_pca = pca.transform(test_emb)

pca_cols = [f"pca_{i}" for i in range(PCA_DIM)]

# 질문 -> PCA벡터 매핑
train_q_to_pca = {q: v for q, v in zip(train_questions, train_pca)}
test_q_to_pca = {q: v for q, v in zip(test_questions, test_pca)}

# 데이터프레임에 PCA 피처 추가
train_df[pca_cols] = pd.DataFrame(train_df[TEXT_COL].map(train_q_to_pca).tolist(), index=train_df.index)
test_df[pca_cols] = pd.DataFrame(test_df[TEXT_COL].map(test_q_to_pca).tolist(), index=test_df.index)

train_df[pca_cols].head(3)


Step 3: SBERT 임베딩 및 PCA 진행 중...


,pca_0,pca_1,pca_2,pca_3,pca_4
8727,3.396134,3.795578,-3.064728,0.571887,-0.245574
8738,3.396134,3.795578,-3.064728,0.571887,-0.245574
8739,3.396134,3.795578,-3.064728,0.571887,-0.245574


In [5]:
print("Step 4: KMeans 질문 클러스터링 진행 중...")

kmeans = KMeans(n_clusters = N_CLUSTERS, random_state=42, n_init=10)

# KMeans는 train 질문 임베딩으로만 fit
train_cluster = kmeans.fit_predict(train_emb)
test_cluster = kmeans.predict(test_emb)

# 질문 -> cluster 매핑
train_q_to_cluster = {q: c for q, c in zip(train_questions, train_cluster)}
test_q_to_cluster = {q: c for q, c in zip(test_questions, test_cluster)}

# 데이터프레임에 q_cluster 추가
train_df["q_cluster"] = train_df[TEXT_COL].map(train_q_to_cluster).astype(int)
test_df["q_cluster"] = test_df[TEXT_COL].map(test_q_to_cluster).astype(int)

train_df[["q_cluster"]].head(3)


Step 4: KMeans 질문 클러스터링 진행 중...


,q_cluster
8727,2
8738,2
8739,2


In [6]:
if USE_TFIDF_SVD:
    print("Step 5: TF-IDF + SVD 진행 중...")

    tfidf = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, ngram_range=(1, 2))
    train_tfidf = tfidf.fit_transform(train_df[TEXT_COL])
    test_tfidf = tfidf.transform(test_df[TEXT_COL])

    svd = TruncatedSVD(n_components=SVD_DIM, random_state=42)
    train_svd = svd.fit_transform(train_tfidf)
    test_svd = svd.transform(test_tfidf)

    svd_cols = [f"svd_{i}" for i in range(SVD_DIM)]

    train_df[svd_cols] = pd.DataFrame(train_svd, index=train_df.index, columns=svd_cols)
    test_df[svd_cols] = pd.DataFrame(test_svd, index=test_df.index, columns=svd_cols)

    print("SVD 피처 추가 완료:", len(svd_cols), "개")
else:
    svd_cols = []
    print("TF-IDF + SVD 옵션이 꺼져있습니다.")


Step 5: TF-IDF + SVD 진행 중...
SVD 피처 추가 완료: 50 개


In [ ]:
print("Step 7: 최종 피처 구성 및 group 배열 생성 중...")

feature_cols = (
    scaled_score_cols
    + ["league_encoded", "q_cluster"]
    + pca_cols
    + svd_cols
    + team_encoded_cols
)

X_train = train_df[feature_cols]
y_train = train_df["relevance"].values

X_test = test_df[feature_cols]
y_test = test_df["relevance"].values

# 그룹(질문별 후보 개수)
group_train = train_df.groupby(GROUP_COL, sort=False).size().values
group_test = test_df.groupby(GROUP_COL, sort=False).size().values

categorical_features = ["league_encoded", "q_cluster"] + team_encoded_cols

print("피처 수:", len(feature_cols))
print("X_train:", X_train.shape, " / X_test:", X_test.shape)


In [ ]:
print("Step 8: LGBMRanker 학습 시작...")

ranker = lgb.LGBMRanker(**LGB_PARAMS)

ranker.fit(
    X_train, y_train,
    group=group_train,
    eval_set=[(X_test, y_test)],
    eval_group=[group_test],
    eval_at=[1, 3, 5],
    categorical_feature=categorical_features
)


In [ ]:
print("Step 9: 성능 평가(질문별 Mean NDCG) 진행 중...")

preds = ranker.predict(X_test)
test_df["preds"] = preds

ndcg_list = []
for q, q_subset in test_df.groupby(GROUP_COL):
    if len(q_subset) > 1:
        score = ndcg_score([q_subset["relevance"]], [q_subset["preds"]])
        ndcg_list.append(score)

print("\n" + "=" * 35)
print(f"✅ Mean NDCG: {np.mean(ndcg_list):.4f}")
print("=" * 35)

# (선택) 어떤 피처를 썼는지 확인
feature_cols


In [ ]:
print("Step 10: GridSearch(수동 루프) 시작...")

# ------------------------------------------------------------
# 1) 질문별 Mean NDCG 계산 함수
# ------------------------------------------------------------
def mean_ndcg_by_group(df_with_pred, group_col=GROUP_COL, y_col="relevance", pred_col="preds"):
    """
    df_with_pred: pred_col(예측점수)가 들어있는 데이터프레임
    group_col: 질문 그룹 컬럼명
    y_col: 정답 컬럼명 (relevance)
    pred_col: 예측점수 컬럼명 (preds)
    """
    ndcg_list = []
    
    # 질문 단위로 묶어서 NDCG 계산
    for q, subset in df_with_pred.groupby(group_col):
        # 후보가 2개 이상일 때만 NDCG 의미가 있음
        if len(subset) > 1:
            score = ndcg_score([subset[y_col]], [subset[pred_col]])
            ndcg_list.append(score)
    
    # 유효 질문이 없으면 0 반환
    if len(ndcg_list) == 0:
        return 0.0
    
    return float(np.mean(ndcg_list))


# ------------------------------------------------------------
# 2) 그리드(탐색할 파라미터 후보들) 정의
#    - 너무 넓게 잡으면 오래 걸리니까, 초보자는 작게 시작 추천
# ------------------------------------------------------------
param_grid = {
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "n_estimators": [200, 400, 600],
    "num_leaves": [15, 31, 63],
    "min_child_samples": [10, 20, 40],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# ------------------------------------------------------------
# 3) 최고 성능 저장 변수
# ------------------------------------------------------------
best_score = -1
best_params = None
best_model = None

# ------------------------------------------------------------
# 4) 조합 개수 계산(대략적인 규모 확인용)
# ------------------------------------------------------------
total = 1
for k, v in param_grid.items():
    total *= len(v)
print(f"총 조합 수: {total} (조합이 너무 크면 리스트를 줄여줘!)")


# ------------------------------------------------------------
# 5) GridSearch 실행(수동 for-loop)
# ------------------------------------------------------------
cnt = 0

for lr in param_grid["learning_rate"]:
    for md in param_grid["max_depth"]:
        for ne in param_grid["n_estimators"]:
            for nl in param_grid["num_leaves"]:
                for mcs in param_grid["min_child_samples"]:
                    for ss in param_grid["subsample"]:
                        for cs in param_grid["colsample_bytree"]:
                            
                            cnt += 1
                            
                            # (1) 모델 생성: 현재 조합으로 파라미터 세팅
                            params = dict(LGB_PARAMS)  # 기본 파라미터 복사
                            params.update({
                                "learning_rate": lr,
                                "max_depth": md,
                                "n_estimators": ne,
                                "num_leaves": nl,
                                "min_child_samples": mcs,
                                "subsample": ss,
                                "colsample_bytree": cs,
                            })
                            
                            model = lgb.LGBMRanker(**params)
                            
                            # (2) 학습: group_train이 반드시 필요
                            model.fit(
                                X_train, y_train,
                                group=group_train,
                                categorical_feature=categorical_features,
                            )
                            
                            # (3) 예측
                            preds = model.predict(X_test)
                            
                            # (4) 평가를 위해 test_df 복사본에 preds 저장
                            tmp = test_df[[GROUP_COL, "relevance"]].copy()
                            tmp["preds"] = preds
                            
                            # (5) 질문별 Mean NDCG 계산
                            score = mean_ndcg_by_group(tmp, group_col=GROUP_COL, y_col="relevance", pred_col="preds")
                            
                            # (6) 진행 로그(너무 많으면 print 줄여도 됨)
                            print(f"[{cnt}/{total}] lr={lr}, md={md}, ne={ne}, leaves={nl}, mcs={mcs}, ss={ss}, cs={cs} -> MeanNDCG={score:.4f}")
                            
                            # (7) 최고 성능 갱신
                            if score > best_score:
                                best_score = score
                                best_params = params
                                best_model = model

print("\n" + "=" * 50)
print(f"✅ Best Mean NDCG: {best_score:.4f}")
print("✅ Best Params:")
for k, v in best_params.items():
    # 너무 길어지는 파라미터는 기본값이 많으니 핵심만 보고 싶으면 여기 필터링 가능
    print(f"  - {k}: {v}")
print("=" * 50)


In [ ]:
print("Step 11: Best 모델 최종 평가...")

# best_model이 이미 X_train으로 학습된 상태지만,
# 혹시 모르니 best_params로 새로 만들고 다시 학습하는 방식(재현성 좋음)
final_model = lgb.LGBMRanker(**best_params)

final_model.fit(
    X_train, y_train,
    group=group_train,
    categorical_feature=categorical_features,
)

final_preds = final_model.predict(X_test)

final_tmp = test_df[[GROUP_COL, "relevance"]].copy()
final_tmp["preds"] = final_preds

final_score = mean_ndcg_by_group(final_tmp, group_col=GROUP_COL, y_col="relevance", pred_col="preds")

print("\n" + "=" * 40)
print(f"✅ 최종 Best Mean NDCG: {final_score:.4f}")
print("=" * 40)
